In [7]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import getcwd
import numpy as np
from PIL import Image
import shutil
import matplotlib.pyplot as plt
import os.path
from xml.etree.ElementTree import parse, Element
import imgaug as ia
from imgaug import augmenters as iaa


ia.seed(1)


def read_xml_annotation(root, image_id):
    in_file = open(os.path.join(root, image_id))
    tree = ET.parse(in_file)
    root = tree.getroot()
    bndboxlist = []

    for object in root.findall('object'):  # 找到root节点下的所有country节点
        bndbox = object.find('bndbox')  # 子节点下节点rank的值

        xmin = int(bndbox.find('xmin').text)
        xmax = int(bndbox.find('xmax').text)
        ymin = int(bndbox.find('ymin').text)
        ymax = int(bndbox.find('ymax').text)
        # print(xmin,ymin,xmax,ymax)
        bndboxlist.append([xmin, ymin, xmax, ymax])
        # print(bndboxlist)

    bndbox = root.find('object').find('bndbox')
    return bndboxlist


# (506.0000, 330.0000, 528.0000, 348.0000) -> (520.4747, 381.5080, 540.5596, 398.6603)
def change_xml_annotation(root, image_id, new_target):
    new_xmin = new_target[0]
    new_ymin = new_target[1]
    new_xmax = new_target[2]
    new_ymax = new_target[3]

    in_file = open(os.path.join(root, str(image_id) + '.xml'))  # 这里root分别由两个意思
    tree = ET.parse(in_file)
    xmlroot = tree.getroot()
    object = xmlroot.find('object')
    bndbox = object.find('bndbox')
    xmin = bndbox.find('xmin')
    xmin.text = str(new_xmin)
    ymin = bndbox.find('ymin')
    ymin.text = str(new_ymin)
    xmax = bndbox.find('xmax')
    xmax.text = str(new_xmax)
    ymax = bndbox.find('ymax')
    ymax.text = str(new_ymax)
    tree.write(os.path.join(root, str("%06d" % (str(id) + '.xml'))))


def change_xml_list_annotation(root, image_id, new_target, saveroot, id):
    in_file = open(os.path.join(root, str(image_id) + '.xml'))  # 这里root分别由两个意思
    tree = ET.parse(in_file)
    elem = tree.find('filename')
    elem.text = (str("%06d" % int(id)) + '.jpg')
    xmlroot = tree.getroot()
    index = 0

    for object in xmlroot.findall('object'):  # 找到root节点下的所有country节点
        bndbox = object.find('bndbox')  # 子节点下节点rank的值

        # xmin = int(bndbox.find('xmin').text)
        # xmax = int(bndbox.find('xmax').text)
        # ymin = int(bndbox.find('ymin').text)
        # ymax = int(bndbox.find('ymax').text)

        new_xmin = new_target[index][0]
        new_ymin = new_target[index][1]
        new_xmax = new_target[index][2]
        new_ymax = new_target[index][3]

        xmin = bndbox.find('xmin')
        xmin.text = str(new_xmin)
        ymin = bndbox.find('ymin')
        ymin.text = str(new_ymin)
        xmax = bndbox.find('xmax')
        xmax.text = str(new_xmax)
        ymax = bndbox.find('ymax')
        ymax.text = str(new_ymax)

        index = index + 1

    tree.write(os.path.join(saveroot, str("%06d" % int(id)) + '.xml'))
    #print(str("%06d" % int(id)) + '.xml')


def mkdir(path):
    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")
    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在')
        return False


if __name__ == "__main__":

    IMG_DIR = "/notebooks/storage/OKK/OKK/testfile/jpg"
    XML_DIR = "/notebooks/storage/OKK/OKK/testfile/annotation"

    AUG_XML_DIR = "/notebooks/storage/experiment/VOCdevkit/annotations/"  # 存储增强后的XML文件夹路径
    try:
        shutil.rmtree(AUG_XML_DIR)
    except FileNotFoundError as e:
        a = 1
    mkdir(AUG_XML_DIR)

    AUG_IMG_DIR = "/notebooks/storage/experiment/VOCdevkit/jPEGImages/"  # 存储增强后的影像文件夹路径
    try:
        shutil.rmtree(AUG_IMG_DIR)
    except FileNotFoundError as e:
        a = 1
    mkdir(AUG_IMG_DIR)

    #AUGLOOP = 1  # 每张影像增强的数量
    #filename = 9710
    filename = 16000
    boxes_img_aug_list = []
    new_bndbox = []
    new_bndbox_list = []

    # 影像增强
    seq = iaa.Sequential([
        iaa.Flipud(0.5),  # vertically flip 20% of all images
        iaa.Fliplr(0.5),  # 镜像
        #iaa.Multiply((1.2, 1.5)),  # change brightness, doesn't affect BBs
        iaa.GaussianBlur(sigma=(0, 3.0)),  # iaa.GaussianBlur(0.5),
        iaa.Affine(
            translate_px={"x": 15, "y": 15},
            scale=(0.8, 0.95),
            rotate=(-30, 30)
        ),  # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
        #iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
        #iaa.EdgeDetect(alpha=(0.5, 1.0)),
        #iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
        #iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)), # sharpen images
        #iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
        #iaa.Grayscale(alpha=(0.0, 1.0))
        
    ])
    
    for root, sub_folders, files in os.walk(XML_DIR):

        for name in files:
            print(name)
            if name == ".DS_Store":
                continue
            file_path = os.path.join(XML_DIR, name)
            dom = parse(file_path)
            root = dom.getroot()
            weight = 22
            for obj in root.iter('object'):#获取object节点中的name子节点
                tmp_name = obj.find('name').text
                if tmp_name == 'yslxb': 
                    weight+=6
                if tmp_name == 'zylxb': 
                    weight+=3
                if tmp_name == 'zhylxb': 
                    weight-=2.864
                if tmp_name == 'wylxb': 
                    weight-=0.944
                if tmp_name == 'gzlxb': 
                    weight -=3.52
                if tmp_name == 'fylxb': 
                    weight -=1.424
            
            
            
            
            
            
            print('该文件增强权重',weight)
            bndbox = read_xml_annotation(XML_DIR, name)
            shutil.copy(os.path.join(XML_DIR, name), AUG_XML_DIR)
            shutil.copy(os.path.join(IMG_DIR, name[:-4] + '.jpg'), AUG_IMG_DIR)#复制文件
            for epoch in range(int(weight)):
                
                seq_det = seq.to_deterministic()  # 保持坐标和图像同步改变，而不是随机
                # 读取图片
                img = Image.open(os.path.join(IMG_DIR, name[:-4] + '.jpg'))
                # sp = img.size
                img = np.asarray(img)
                # bndbox 坐标增强
                for i in range(len(bndbox)):
                    bbs = ia.BoundingBoxesOnImage([
                        ia.BoundingBox(x1=bndbox[i][0], y1=bndbox[i][1], x2=bndbox[i][2], y2=bndbox[i][3]),
                    ], shape=img.shape)

                    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
                    boxes_img_aug_list.append(bbs_aug)

                    # new_bndbox_list:[[x1,y1,x2,y2],...[],[]]
                    n_x1 = int(max(1, min(img.shape[1], bbs_aug.bounding_boxes[0].x1)))
                    n_y1 = int(max(1, min(img.shape[0], bbs_aug.bounding_boxes[0].y1)))
                    n_x2 = int(max(1, min(img.shape[1], bbs_aug.bounding_boxes[0].x2)))
                    n_y2 = int(max(1, min(img.shape[0], bbs_aug.bounding_boxes[0].y2)))
                    if n_x1 == 1 and n_x1 == n_x2:
                        n_x2 += 1
                    if n_y1 == 1 and n_y2 == n_y1:
                        n_y2 += 1
                    if n_x1 >= n_x2 or n_y1 >= n_y2:
                        print('error', name)
                    new_bndbox_list.append([n_x1, n_y1, n_x2, n_y2])
                # 存储变化后的图片
                
                image_aug = seq_det.augment_images([img])[0]
                path = os.path.join(AUG_IMG_DIR,#新path
                                    str("%06d" % filename) + '.jpg')
                image_auged = bbs.draw_on_image(image_aug, thickness=0)
                Image.fromarray(image_auged).save(path)

                # 存储变化后的XML
                change_xml_list_annotation(XML_DIR, name[:-4], new_bndbox_list, AUG_XML_DIR,
                                           filename)
                #print(str("%06d" % filename) + '.jpg')
                filename+=1
                new_bndbox_list = []


/notebooks/storage/experiment/VOCdevkit/annotations/ 创建成功
/notebooks/storage/experiment/VOCdevkit/jPEGImages/ 创建成功
151.xml
该文件增强权重 15.632000000000001
305.xml
该文件增强权重 10.152000000000001
386.xml
该文件增强权重 20.264000000000003
431.xml
该文件增强权重 14.152000000000001
110.xml
该文件增强权重 34
285.xml
该文件增强权重 28.632
252.xml
该文件增强权重 11.808
121.xml
该文件增强权重 13.264000000000003
376.xml
该文件增强权重 15.328
369.xml
该文件增强权重 5.712
38.xml
该文件增强权重 7.023999999999997
25.xml
该文件增强权重 27.192
130.xml
该文件增强权重 17.248
159.xml
该文件增强权重 12.096
463.xml
该文件增强权重 18.208000000000002
312.xml
该文件增强权重 13.056000000000001
402.xml
该文件增强权重 14.864
482.xml
该文件增强权重 9.728000000000002
79.xml
该文件增强权重 10.496000000000002
70.xml
该文件增强权重 28
100.xml
该文件增强权重 15.407999999999998
150.xml
该文件增强权重 21.328
318.xml
该文件增强权重 19.288
411.xml
该文件增强权重 6.416000000000004


In [9]:
import os

def file_name(file_dir):
    a = 0
    m = 0
    for root, dirs, files in os.walk(file_dir):
        for i in files:
            m+=1
            if m == int(len(files)/2):
               print("***************************")
            print(i.replace(".jpg",""))
    print(m)
                
file_name("/notebooks/storage/experiment/VOCdevkit/jPEGImages/")

016022
016197
016250
016033
016186
016137
016179
016311
016349
016170
016330
016315
016297
016026
016090
016206
312
016241
016047
016089
016080
016168
016230
016159
016192
016336
016093
016111
016236
016204
016040
016085
016363
016057
016018
79
016098
016054
016348
016314
016120
016045
016027
016029
016191
016278
016128
016219
016147
016104
121
016062
016213
016253
016196
016094
016328
016002
016380
016373
016141
016188
016340
016282
016242
016078
016210
016142
016165
016257
016254
016324
016224
016087
016162
016112
016155
016202
016214
016265
016195
016016
110
252
016058
016377
016193
016039
016332
016246
016025
016017
016055
016113
016288
016310
016011
016077
016302
016059
016021
016154
016181
016290
016331
016185
016012
016231
016035
016243
016183
016284
016153
016329
016342
016003
016013
016171
016000
016143
016096
016368
016145
016344
016259
016124
016042
016060
016299
016294
016359
016063
016234
016115
016101
38
016198
016180
016341
016248
016167
016286
151
016239
016068
016099
0